In [1]:
from tqdm import tqdm

import numpy as np

import sgkit as sg
from sgkit.io.vcf import vcf_to_zarr

import tszip

import sys
sys.path.append("../../tsimpute/src/")
import compare_vcfs as cv
import impute_by_sample_matching as impute
import util


### Run tskit.lshmm


In [2]:
trees_dir = "../data/trees/"
zarr_dir = "../data/zarr/"
vcf_dir = "../data/vcf/"
lshmm_dir = "../analysis/lshmm/"


In [3]:
ref_ts_file = trees_dir + "ref.tsz"
ref_ts = tszip.decompress(ref_ts_file)


In [4]:
target_chip_ds_compat = sg.load_dataset(zarr_dir + "target_chip_compat.zarr")
target_chip_ds_compat


<xarray.Dataset>
Dimensions:             (variants: 7899, samples: 176, ploidy: 2, contigs: 1,
                         alleles: 4)
Dimensions without coordinates: variants, samples, ploidy, contigs, alleles
Data variables:
    call_genotype       (variants, samples, ploidy) float64 dask.array<chunksize=(1975, 44, 1), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(3950, 88, 1), meta=np.ndarray>
    contig_id           (contigs) <U2 dask.array<chunksize=(1,), meta=np.ndarray>
    sample_id           (samples) object dask.array<chunksize=(176,), meta=np.ndarray>
    variant_allele      (variants, alleles) <U1 dask.array<chunksize=(7899, 4), meta=np.ndarray>
    variant_contig      (variants) int8 dask.array<chunksize=(7899,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(7899,), meta=np.ndarray>
Attributes:
    contigs:  ['20']
    source:   sgkit-0.7.0

In [5]:
switch_prob = np.repeat(1e-08, ref_ts.num_sites)
mismatch_prob = np.repeat(1e-20, ref_ts.num_sites)
precision = 25


In [6]:
h1, h2, h3 = impute.impute_by_sample_matching(
    ref_ts=ref_ts,
    target_ds=target_chip_ds_compat,
    switch_prob=switch_prob,
    mismatch_prob=mismatch_prob,
    precision=precision,
)
print(h2.shape)


100%|██████████| 504314/504314 [00:04<00:00, 125623.80it/s]

(352, 504314)


In [7]:
np.save(lshmm_dir + "h1" + "_" + "p" + str(precision) + ".npy", h1)
np.save(lshmm_dir + "h2" + "_" + "p" + str(precision) + ".npy", h2)
np.save(lshmm_dir + "h3" + "_" + "p" + str(precision) + ".npy", h3)


In [9]:
h2 = np.load(lshmm_dir + "h2" + "_" + "p" + str(precision) + ".npy")
assert h2.shape == (
    target_chip_ds_compat.dims['samples'] * target_chip_ds_compat.dims['ploidy'],
    ref_ts.num_sites,
)


In [13]:
target_ds = sg.load_dataset(zarr_dir + "target.zarr")
sample_names = target_ds.vcf_header.split("\n")[5].split("\t")[9:]
lshmm_ts = util.add_individuals_to_tree_sequence(ref_ts, h2, sample_names)
lshmm_ts


100%|██████████| 176/176 [00:02<00:00, 74.27it/s]


In [ ]:
out_prefix = "target_lshmm"


In [14]:
lshmm_ts_file = lshmm_dir + out_prefix + ".tsz"
tszip.compress(lshmm_ts, lshmm_ts_file)


In [15]:
lshmm_vcf_file = vcf_dir + out_prefix + ".vcf"

assert ref_ts.num_individuals == lshmm_ts.num_individuals - len(sample_names)

with open(lshmm_vcf_file, "w") as f:
    lshmm_ts.write_vcf(
        output=f,
        contig_id="20",
        individuals=np.arange(
            ref_ts.num_individuals,
            lshmm_ts.num_individuals
        ),
        individual_names=sample_names,
    )


In [16]:
%%bash
vcf_dir="../data/vcf/"
out_prefix=${vcf_dir}"target_lshmm"
bgzip ${out_prefix}".vcf"
bcftools index ${out_prefix}".vcf.gz"


In [17]:
lshmm_vcf_file = vcf_dir + out_prefix + ".vcf.gz"
vcf_to_zarr(lshmm_vcf_file, zarr_dir + out_prefix + ".zarr")
lshmm_ds = sg.load_dataset(zarr_dir + out_prefix + ".zarr")


In [18]:
lshmm_ds_compat = cv.remap_to_acgt(lshmm_ds, num_workers=6)
sg.save_dataset(lshmm_ds_compat, zarr_dir + out_prefix + "_compat.zarr")


100%|██████████| 504314/504314 [53:52<00:00, 156.01it/s] 
